In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

from transformers import Trainer, TrainingArguments

from tqdm.auto import tqdm

import pandas as pd
import numpy as np


In [ ]:
model_name = 'gpt2'

In [ ]:
model_save_path = './diabatasty'

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embedding(50260, 768)

In [ ]:
tokenizer.save_pretrained(model_save_path)

('./diabatasty/tokenizer_config.json',
 './diabatasty/special_tokens_map.json',
 './diabatasty/vocab.json',
 './diabatasty/merges.txt',
 './diabatasty/added_tokens.json',
 './diabatasty/tokenizer.json')

In [ ]:
tokenizer.convert_tokens_to_ids(['<|pad|>'])

[50259]

In [ ]:
def generate(prompt):
    inputs = tokenizer.encode_plus(prompt, return_tensors='pt')
    output = model.generate(**inputs,max_length=256,do_sample=True,pad_token_id=50259)
    print(tokenizer.decode(output[0]))

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|unknown|>',
 'pad_token': '<|pad|>'}

In [ ]:
tokenizer.convert_tokens_to_ids(['<|startoftext|>'],)

[50257]

In [ ]:
clean = pd.read_csv('/content/Cleaned_Indian_Food_Dataset.csv')
clean = clean.sample(frac=1)
clean.reset_index(drop=True,inplace=True)
clean.rename(columns={'Cleaned-Ingredients': 'ingredients', 'TranslatedInstructions': 'instructions'}, inplace=True)

clean.head()

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,instructions,URL,ingredients,image-url,Ingredient-count
0,Keema Bhindi Masala Recipe - Okra & Minced Mea...,250 grams Bhindi (Lady Finger/Okra) - cut into...,45,North Indian Recipes,To begin making the Keema Bhindi Masala Recipe...,https://www.archanaskitchen.com/keema-bhindi-r...,"tomato,salt,coriander (dhania),ginger,mustard ...",https://www.archanaskitchen.com/images/archana...,15
1,Vegetarian Grilled Gyros Recipe,1 cup Button mushrooms - (use portabella mushr...,55,Greek,To begin making the Vegetarian Grilled Gyros r...,https://www.archanaskitchen.com/vegetarian-gri...,"yellow bell pepper (capsicum),tomato,red bell ...",https://www.archanaskitchen.com/images/archana...,17
2,Whole Wheat Palak Naan Recipe,"1 cup Curd (Dahi / Yogurt),1 teaspoon Baking s...",50,North Indian Recipes,"To begin making the Whole Wheat Palak Naan, fi...",https://www.archanaskitchen.com/whole-wheat-pa...,"salt,wheat flour,kalonji (onion nigella seeds)...",https://www.archanaskitchen.com/images/archana...,9
3,Savoury Yogurt Parfait Recipe,Mint Leaves (Pudina) - chopped (you can also u...,20,Continental,To begin making the Savoury Yogurt Parfait Rec...,https://www.archanaskitchen.com/savoury-yogurt...,mint leaves (pudina) (you can also basilcorian...,https://www.archanaskitchen.com/images/archana...,13
4,Cheesy Veg Paniyaram Recipe With Cheesy Tomato...,"2 cups Bajra Dosa Batter,1 Carrot (Gajjar) - g...",25,South Indian Recipes,To begin making Cheesy Veg Paniyaram Recipe fi...,https://www.archanaskitchen.com/cheesy-veg-pan...,"tomato,salt,sunflower oil,green chilli,bajra d...",https://www.archanaskitchen.com/images/archana...,16


In [ ]:
def print_recipe(idx):
    print(f"{clean['ingredients'][idx]}\n\n{clean['instructions'][idx]}")

In [ ]:
def form_string(ingredient,instruction):
    s = f"<|startoftext|>Ingredients:\n{ingredient.strip()}\n\nInstructions:\n{instruction.strip()}<|endoftext|>"
    return s

In [ ]:
clean = clean.sample(n=100, random_state=1)

In [ ]:
data = clean.apply(lambda x:form_string(x['ingredients'],x['instructions']),axis=1).to_list()

https://towardsdatascience.com/guide-to-fine-tuning-text-generation-models-gpt-2-gpt-neo-and-t5-dc5de6b3bc5e

In [ ]:
train_size = 0.85
train_len = int(train_size * len(data))
train_data = data[:train_len]
val_data = data[train_len:]

In [ ]:
class RecipeDataset:
    def __init__(self,data):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for item in tqdm(data):
            encodings = tokenizer.encode_plus(item,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=1024,
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
def collate_fn(batch):
    return {
        'input_ids': torch.stack([item[0] for item in batch]),
        'attention_mask': torch.stack([item[1] for item in batch]),
        'labels': torch.stack([item[0] for item in batch])
    }

In [ ]:
train_ds = RecipeDataset(train_data)
val_ds = RecipeDataset(val_data)

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
args = TrainingArguments(output_dir=model_save_path,
                         per_device_train_batch_size=2,
                         per_device_eval_batch_size=2,
                         gradient_accumulation_steps=2,
                         report_to='none',
                         num_train_epochs=1,
                         save_strategy='no'
                        )

In [ ]:
optim = torch.optim.AdamW(model.parameters(),lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,20,eta_min=1e-7)

In [ ]:
trainer = Trainer(model,
                  args,
                  train_dataset=train_ds,
                  eval_dataset=val_ds,
                  data_collator=collate_fn,
                  optimizers=(optim,scheduler)
                 )

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

Saving model checkpoint to ./khaanaGPT
Configuration saved in ./khaanaGPT/config.json
Model weights saved in ./khaanaGPT/pytorch_model.bin


Dissco module

In [ ]:
def calculate_dissco(age, gender, hba1c, conditions):
    # Gender factor
    gender_factor = 1.4 if gender.lower() == 'female' else 1.0

    # Base HDF score
    hdf = 1.0

    # Define condition impacts
    condition_impacts = {
        "hypertension": 0.5,
        "cardiomyopathy": 1,
        "pericardial_disease": 1,
        "myocarditis": 1,
        "coronary_heart_disease": 1,
        "peripheral_vascular_disease": 2,
        "stent_or_bypass": 3
    }

    # Add factors for each present condition
    for condition in conditions:
        hdf += condition_impacts.get(condition.lower(), 0)  # Ignore unrecognized conditions

    # Calculate raw DissCo score
    dissco_score = ((age / 10) + hba1c) * gender_factor * hdf

    # Normalize the score (max possible score is 34.3)
    max_score = 34.3
    normalized_dissco = dissco_score / max_score

    return normalized_dissco

# Example usage:
age = 60
gender = "male"
hba1c = 6
conditions = ["hypertension", "stent or bypass"]

normalized_dissco_score = calculate_dissco(age, gender, hba1c, conditions)
print("Normalized DissCo Score:", normalized_dissco_score)

Normalized DissCo Score: 0.5247813411078718


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import pipeline

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config

In [ ]:
pl = pipeline(task='text-generation',model='/content/drive/MyDrive/Colab Notebooks/assets/model', from_pt=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import torch

try:
    state_dict = torch.load('/content/sample_data/model/pytorch_model.bin')
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")

Error loading model: [Errno 2] No such file or directory: '/content/sample_data/model/pytorch_model.bin'


<ipython-input-24-e4190ea1d5ba>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/content/sample_data/model/pytorch_model.bin')


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
import torch
# Load the configuration from the config.json file
config = GPT2Config.from_pretrained('/content/drive/MyDrive/Colab Notebooks/assets/model/config.json')  # Specify the path to your config file

# Load the tokenizer (adjust based on your model, e.g., GPT2 or custom)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # Or a custom tokenizer if applicable

# Load the model architecture and weights from the .bin file
model = GPT2LMHeadModel.from_pretrained(
    '/content/drive/MyDrive/Colab Notebooks/assets/model',  # Replace with the path to the directory containing the .bin file
    config=config
)
pl = pipeline(task='text-generation', model=model, tokenizer=tokenizer)



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
prompt = "potato, cheese, ginger"
result = pl(prompt,max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259)

print(result[0]['generated_text'])

potato, cheese, ginger and oregano.

Ingredients:
salt
cumin powder (jeera)
red bell pepper (capsicum)
ginger
mustard seeds
asafoetida (hing)
curry leaves
coriander (dhania) seeds
cumin powder (jeera)
green chillies
turmeric powder

Instructions:
wash and wash the bell peppers and coriander leaves.heat oil in a heavy bottomed pan.
add mustard seeds and allow it crackle.add curry leaves, red capsicum and allow it splutter.once the curry leaves have spluttered add asafoetida and saute for a minute and turn off the heat.
allow the curry leaves to soften and add the chopped coriander leaves.now add the grated ginger, green chillies, turmeric powder, cumin powder, cumin powder, red chillies, coriander leaves.mix well and let the mixture simmer for about 30 seconds.after a few minutes, turn off the heat and allow the mixture  add the chopped coriander leaves, sprinkle some salt and mix well.
let it simmer for a few more minutes.once the gravy is well combined add the chopped chilli and mix w

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
import torch
# Load the configuration from the config.json file
config = GPT2Config.from_pretrained('/content/sample_data/model/config.json')  # Specify the path to your config file

# Load the tokenizer (adjust based on your model, e.g., GPT2 or custom)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # Or a custom tokenizer if applicable

# Load the model architecture and weights from the .bin file
# Try loading with 'latin1' encoding to handle potential encoding issues
try:
    model = GPT2LMHeadModel.from_pretrained(
        '/content/sample_data/model',  # Replace with the path to the directory containing the .bin file
        config=config,
        # Specify 'latin1' encoding
        _fast_init=False  # Force the model to load weights using torch.load
    )
except OSError as e:
    if "Unable to load weights" in str(e):
        print("Trying to load with 'latin1' encoding...")
        model = GPT2LMHeadModel.from_pretrained(
            '/content/sample_data/model',
            config=config,
            _fast_init=False,  # Force the model to load weights using torch.load
            torch_dtype=torch.float32 # Setting torch_dtype
        )
    else:
        raise e

pl = pipeline(task='text-generation', model=model, tokenizer=tokenizer)

Trying to load with 'latin1' encoding...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


OSError: Unable to load weights from pytorch checkpoint file for '/content/sample_data/model/pytorch_model.bin' at '/content/sample_data/model/pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [ ]:
def create_prompt(ingredients):
    ingredients = ','.join([x.strip().lower() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',','\n')
    s = f"<|startoftext|>Ingredients:\n{ingredients}\n"
    return s

In [ ]:
ingredients = ['Rice, Potatoes, Tomatoes']

In [ ]:

for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

ValueError: The following `model_kwargs` are not used by the model: ['from_pt'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        # Remove any unintended 'from_pt' argument from kwargs
        )[0]['generated_text']) # Accessing the generated text from the output

ValueError: The following `model_kwargs` are not used by the model: ['from_pt'] (note: typos in the generate arguments will also show up in this list)